<a href="https://colab.research.google.com/github/PatrickMarinich/Mario-Kart-Stats-Project/blob/main/Kartnite_Stats_v2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kartnite Stats** v2.1

*Written by: Patrick Marinich*

**Goal:** This scripts goal is to take user inputted *Mario Kart Wii* races and other stats and save them to a database to be used for calculations and analysis when user requested. 

**Current Functionalities**


*   Take in a user inputted race, with the track, players, and scores, and adds them to the database of all scores
*   Keep track of how many times each player playes on a singular track
* Get the average score per track of all of the players
* Determine which player is the best on any specific track
* A leaderboard to display the ordering of player seeding (power rankings)
* The ability to search for all the stats of a specifc track, including total points, player averages, and the individual track leaderboards
* The ability to search of all of the stats of a individual player, including total points, track averages, Track MVPs, and total power ranking points (Kart Score)
* The ability to correct mistakes in entering race points, by having a user setting to manually edit the points of a specific player
* Ability to view the database containing all of the data





**Important:** Make sure to run all cells before running the main cell, the main cell is the last one. If all function cells are not run then the main cell will break and throw errors



**Patch Notes**

v2.1 (Jan 4 2022)
- Optimization
- Changes to 'Kart Score' 
- MVP Leaderboards Per Track

v2.0 (Jan 3 2022)
 - Adds Shock Dodges 
 - Adds GP Wins
 - Adds Total Race Count
 - Leaderboards

v1.1 (Jan 2nd 2022)
 - Moved the Main Method into a private method, this reduced the need for scrolling

v1.0 (December 27 2021 - December 31st 2021)
 - Importing and Saving Race Data
 - Editing singular race scores incase of mistakes
 - Viewing Track Records + Best Player on the Track
 - Viewing Player Records


In [97]:
def inputRace(currentData,RaceCount,dfKartScore, Track, Racers, Scores,TrackIndex):  
  #checks to see if the track exists
  if Track in set(currentData["Tracks x Players"]):
    print('Valid', Track)
    #seperates the players into an array, make sure that all players also exist

    racersArray = Racers.split()
    # checks that all players are valid
    validCount = 0
    for racer in racersArray:
      if(racer in set(currentData.columns.values.tolist())):
          print('Valid', racer)
          validCount = validCount + 1
      else: 
          validCount = validCount - 1
          print('Invalid', racer)

  
    if validCount == len(racersArray):
      print("....Everything is valid, Updating the scores....")

      ##if all players are valid update the scores

      #gets scores into an array
      newScores = Scores.split()
      #changes to integers
      for i in range(len(newScores)):
        newScores[i] = int(newScores[i])
      #Adds new Score to the current score for each racer, uses the track index to find the track
      for i in range(len(racersArray)):
        #increments the score
        currentData.at[TrackIndex[Track],racersArray[i]] =  int(currentData.at[TrackIndex[Track],racersArray[i]]) + newScores[i]
        #increments the count
        RaceCount.at[TrackIndex[Track],racersArray[i]] =  int(RaceCount.at[TrackIndex[Track],racersArray[i]]) + 1
       
        #only gives the points if there is a singular MVP
        if(len(getTrackOwner(currentData,RaceCount, Track, TrackIndex).split()) == 1):
        #gives the track owner 2.5 points per player
          dfKartScore.at[0, getTrackOwner(currentData,RaceCount, Track, TrackIndex)] = int(dfKartScore.at[0, getTrackOwner(currentData,RaceCount, Track, TrackIndex)]) + 2

    else: 
      print('There was an error entering the racers, please try again')
  else:
    print('There was an error entering the track name, please try again:', Track, "was invalid")





In [98]:
def editAScore(dfScores, Track, Racer, Score,TrackIndex):
 #makes sure track is valid
  if Track in set(dfScores["Tracks x Players"]):
    print('Valid', Track)
    # makes sure racer is valid
    if Racer in set(dfScores.columns.values.tolist()):
      print('Valid', Racer)
      #changes a specific score
      dfScores.at[TrackIndex[Track],Racer] =  int(dfScores.at[TrackIndex[Track],Racer]) + int(Score)
      print(Racer, '\'s Score was changed by ', Score, ' on ', Track)
    else:  
      print('There was an error entering the racer, please try again')
  else:
     print('There was an error entering the track name, please try again:', Track, "was invalid")



In [99]:
def getPlayerAverage(dfScores, dfRaceCount, Player, Track, TrackIndex):
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
  
  if Player in set(dfNoTracks.columns.values.tolist()):
    #checks for divide by 0

    RaceCount = int(dfRaceCount.at[TrackIndex[Track],Player])
    Score =  int(dfScores.at[TrackIndex[Track],Player])
   
    if(Score == 0 or RaceCount == 0):
      averageScore = 0
    else: 
      averageScore = Score/RaceCount
  
    

  else:
    averageScore = "ERROR"
    print("There was an internal with Averaging Have Pat re-read code")
  

  return averageScore



In [100]:
def getTrackOwner(dfScores,dfRaceCount, Track, TrackIndex):
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]


  #goes through all players and then find the max score

  currentMaxScore = 0
  currentPlayer = ""
  TrackTotalPoints = 0
  for player in set(dfNoTracks.columns.values.tolist()):
    TrackTotalPoints = TrackTotalPoints + int(dfScores.at[TrackIndex[Track], player])

  for player in set(dfNoTracks.columns.values.tolist()):
    #way to calculate track owner, total points + average
    
    playerScore = ((int(dfScores.at[TrackIndex[Track],player])/TrackTotalPoints)*100) + getPlayerAverage(dfScores,dfRaceCount,player,Track,TrackIndex)
  
    if playerScore> currentMaxScore:
      currentPlayer = player
      currentMaxScore = playerScore
    elif playerScore == currentMaxScore:
      currentPlayer = currentPlayer + ", " + player
    else:
      continue


  return currentPlayer



In [101]:
def GetTrackData(dfScores,dfRaceCount,Track,TrackIndex):
 
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
 
  #make sure that the track exists in the data
  if Track in set(dfScores["Tracks x Players"]):
    print("Stats For: ", Track)
    #now display the data for the track plus any interesting stats
    print("\n", "Total Scores")
    print(dfScores.loc[[TrackIndex[Track]]])

    races = 0
    for player in set(dfNoTracks.columns.values.tolist()):
      races = races + int(dfRaceCount.at[TrackIndex[Track], player])
    print('\nTimes Played:',races )

    #prints player averages. 
    print('\nPlayer Averages:')
    for racer in set(dfNoTracks.columns.values.tolist()):
      print(racer,": ", getPlayerAverage(dfScores,dfRaceCount,racer,Track,TrackIndex))
    print('\n')
    print(getTrackOwner(dfScores, dfRaceCount, Track, TrackIndex), "is the MVP for this track")
    
    print('\nMVP Leaderboard')
    #determine the score for each player within the data
    dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'MVP Points':[]})
    counter = 0
   
    #gets the ownership scores of each player for the track
    TrackTotalPoints = 0
    for player in set(dfNoTracks.columns.values.tolist()):
      TrackTotalPoints = TrackTotalPoints + int(dfScores.at[TrackIndex[Track], player])
    for player in set(dfNoTracks.columns.values.tolist()):
      ownershipScore = ((int(dfScores.at[TrackIndex[Track],player])/TrackTotalPoints)*100) + getPlayerAverage(dfScores,dfRaceCount,player,Track,TrackIndex)


      #puts the score into the datafram
      dfLeaderboard.loc[counter] = [player, ownershipScore]
      counter = counter + 1
      playerScore = 0

    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['MVP Points', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)

  else:
    print('Track Selection Was Invalid')


In [102]:
def getPlayerStats(dfScores,dfRaceCount, dfWins,dfShock, dfKartScore, Player, TrackIndex):
  #prints the total Scores
  dfPlayerScores = dfScores[["Tracks x Players", Player]]
  print(Player, "\'s Total Scores")
  print("\n")
  print(dfPlayerScores)

  #prints Averages
  print("\n")
  print(Player, "\'s Track Averages")
  print("\n")
  for track in TrackIndex:
    print(track, "- ", getPlayerAverage(dfScores,dfRaceCount,Player,track,TrackIndex))



  print("\n")
  #gets all of the tracks that the player owns
  Owns = ''
  for track in TrackIndex:
    if getTrackOwner(dfScores,dfRaceCount,track,TrackIndex) == Player:
      
      if Owns == '':
        Owns = track
      else:
        Owns = Owns + ", " + track

  if(Owns == ''):
    print("Track MVP on: None")
  else:
    print("Tracks MVP on:", Owns)

  #prints total wins
  totalRaces = 0
  for Track in TrackIndex:
    totalRaces = totalRaces + int(dfRaceCount.at[TrackIndex[Track],Player])
  print("Total Races Played:", totalRaces)

  #Gp Wins
  print("Grand Prix Wins:", dfWins.at[0,Player])
  #dodges
  print("Shock Dodges:", dfShock.at[0,Player])

  ####KART SCORE####
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
  KartScore = 0
  #gp wins (+100 per win)
  WinsScore = int(dfWins.at[0,Player])*100
    

  #owner score (+2 per player per race)
  OwnedScore = int(dfKartScore.at[0, Player])
     
      

  #Dodges(+2 per dodge)
  DodgeScore = int(dfShock.at[0,Player])*2
    
  #total points(+1 per point)
  PointsScore = 0
  for track in TrackIndex:
   PointsScore = PointsScore + int(dfScores.at[TrackIndex[track], Player])

  #adds in track averages (+4 per) avg point
  avgScore = 0
  for track in TrackIndex:
    avgScore = avgScore + (int(getPlayerAverage(dfScores,dfRaceCount,Player,track,TrackIndex))*4)
  
  KartScore = WinsScore + OwnedScore + DodgeScore + avgScore  + PointsScore
  print("\n")
  print("Kart Score:", KartScore)
  print('--------------')
  print('GP Win Points:', WinsScore)
  print('Points From Being Track MVP:', OwnedScore)
  print('Race Points:', PointsScore)
  print('Track Avg Points:', avgScore)
  print('Shock Dodge Points:', DodgeScore)




In [103]:
#allows for GP winners to be counted
def enterWinner(dfWins, Player):

  if Player in set(dfWins.columns.values.tolist()):
    
    dfWins.at[0, Player] = int(dfWins.at[0,Player]) + 1
    print(Player, 'has', dfWins.at[0,Player], 'wins')


  else: 
    print(Player, "does not exist within the Database, Please try again")



In [104]:
#allowes for Shock dodges
def enterDodges(dfShock, Player, Count):
  if Player in set(dfShock.columns.values.tolist()):
    
    dfShock.at[0, Player] = int(dfShock.at[0,Player]) + int(Count)
    print(Player, 'has', dfShock.at[0,Player], 'shock dodges')


  else: 
    print(Player, "does not exist within the Database, Please try again")


In [105]:
def getSeedings(dfScores,dfRaceCount,dfWins,dfShock,dfKartScore,TrackIndex):
    
    #determine the score for each player within the data
    dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Kart Score':[]})
    counter = 0
   
    #sets up the scores for each player
    for player in set(dfNoTracks.columns.values.tolist()):
      playerScore = 0
      #gp wins (+100 per win)
      playerScore = int(dfWins.at[0,player])*100
    
      #enters the owned track score (+2 per player per race)
      playerScore = playerScore + int(dfKartScore.at[0, player])


      #Dodges(+2 per dodge)
      playerScore = int(dfShock.at[0,player])*2 + playerScore
    
      #total points(+1 per point)
      points = 0
      for track in TrackIndex:
        points = points + int(dfScores.at[TrackIndex[track], player])
      playerScore = playerScore + points

      #adds in track averages (+4 per) avg point
      avgpoints = 0
      for track in TrackIndex:
        avgpoints = avgpoints + (int(getPlayerAverage(dfScores,dfRaceCount,player,track,TrackIndex))*4)
      playerScore = playerScore + avgpoints

      #puts the score into the datafram
      dfLeaderboard.loc[counter] = [player, playerScore]
      counter = counter + 1
      playerScore = 0

    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Kart Score', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)





In [106]:
def getRaceCountLeaderbaords(dfRaceCount,TrackIndex):
  
    #determine the score for each player within the data
    dfNoTracks = dfRaceCount[dfRaceCount.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Races Played':[]})
    counter = 0
   
    racecount = 0
    #sets up the scores for each player
    for player in set(dfNoTracks.columns.values.tolist()):
      
      for track in TrackIndex:
        racecount = racecount + int(dfRaceCount.at[TrackIndex[track], player])


      #adds the racecount to the leaderboard
      dfLeaderboard.loc[counter] = [player, racecount]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Races Played', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)


In [107]:
def getGPWinsLeaderboard(dfWins):
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Grand Prix Wins':[]})
    counter = 0
    #for each player in wins
    for player in set(dfWins.columns.values.tolist()):
      #adds the wins to the leaderboard
      dfLeaderboard.loc[counter] = [player, dfWins.at[0, player]]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Grand Prix Wins', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)


In [108]:
def getShockDodges(dfShock):
  #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Shock Dodges':[]})
    counter = 0
    #for each player in dodges
    for player in set(dfShock.columns.values.tolist()):
      #adds the dodges to the leaderboard
      dfLeaderboard.loc[counter] = [player, dfShock.at[0, player]]
      counter = counter + 1
      racecount = 0
   
    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Shock Dodges', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)



In [109]:
#This is the main bulk of the code, it culminates all of the previous functions 
#into the user inputted choices, this is the method that gets user inputs and does all 
#of the database scraping and saving for this program to work.
def RunKartniteStats():
### Patrick Marinich December 2021

  #imports
  import pandas as pd
  from google.colab import auth
  auth.authenticate_user()
  import gspread
  from oauth2client.client import GoogleCredentials
  from gspread_dataframe import get_as_dataframe, set_with_dataframe

  # Puts the google sheet into a python object, we will be calling the sh object after a selection is made
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1G0z4XeBjG7Q_Zk_uPtV4v8BPkVT8GD9pmsuk7VIwFKc/edit#gid=0')

  #Track Dictionary, all track names and nicknames go here
  #track indexes are starting at 0 in the mushroom cup ending at 31 at N64 Bowsers castle, +32 for Lava Lake
  TrackIndex = {"Luigi Circuit":0,
  "Moo Moo Meadows":1,
  "Mushroom Gorge":2,
  "Toad's Factory":3,
  "Mario Circuit":4,
  "Coconut Mall":5,
  "DK Summit":6,
  "Wario's Gold Mine":7,
  "Daisy Circuit":8,
  "Koopa Cape":9,
  "Maple Treeway":10,
  "Grumble Volcano":11,
  "Dry Dry Ruins":12,
  "Moonview Highway":13,
  "Bowser's Castle":14,
  "Rainbow Road":15,
  "GCN Peach Beach":16,
  "DS Yoshi Falls":17,
  "SNES Ghost Valley 2":18,
  "N64 Mario Raceway":19,
  "N64 Sherbet Land":20,
  "GBA Shy Guy Beach":21,
  "DS Delfino Square":22,
  "GCN Waluigi Stadium":23,
  "DS Desert Hills":24,
  "GBA Bowser's Castle 3":25,
  "N64 DK's Jungle Parkway":26,
  "GCN Mario Circuit":27,
  "SNES Mario Circuit 3":28,
  "DS Peach Gardens":29,
  "GCN DK Mountain":30,
  "N64 Bowser's Castle":31,
  "Lava Lake":32 }

#for nicknames later currently doesnt work
  # # NickNameIndex= {"Luigi Circuit":0,
  # "Moo Moo Meadows":1,
  # "moo moo":1,
  # "Moo Moo":1,
  # "Mushroom Gorge":2,
  # "Toad's Factory":3,
  # "toads":3,
  # "Toads":3,
  # "Mario Circuit":4,
  # "Coconut Mall":5,
  # "DK Summit":6,
  # "summit":6,
  # "Summit":6,
  # "Wario's Gold Mine":7,
  # "gold mine":7,
  # "Gold Mine":7,
  # "Daisy Circuit":8,
  # "Koopa Cape":9,
  # "Maple Treeway":10,
  # "maple":10,
  # "Maple":10,
  # "Grumble Volcano":11,
  # "grumble":11,
  # "Grumble":11,
  # "Dry Dry Ruins":12,
  # "Dry Dry":12,
  # "dry dry":12,
  # "Moonview Highway":13,
  # "Moonview":13,
  # "moonview":13,
  # "Bowser's Castle":14,
  # "BC Wii":14,
  # "bc wii":14,
  # "BC wii":14,
  # "Rainbow Road":15,
  # "rainbow":15,
  # "Rainbow":15,
  # "GCN Peach Beach":16,
  # "Peach Beach":16,
  # "peach beach":16,
  # "DS Yoshi Falls":17,
  # "yoshi falls":17,
  # "Yoshi Falls":17,
  # "SNES Ghost Valley 2":18,
  # "Ghost Valley":18,
  # "ghost valley":18,
  # "N64 Mario Raceway":19,
  # "N64 Sherbet Land":20,
  # "Sherbet Land":20,
  # "sherbet land":20,
  # "GBA Shy Guy Beach":21,
  # "Shy Guy Beach":21,
  # "shy guy beach":21,
  # "Shy Guy":21,
  # "shy guy":21,
  # "DS Delfino Square":22,
  # "Delfino Square":22,
  # "delfino square":22,
  # "Delfino":22,
  # "delfino":22,
  # "GCN Waluigi Stadium":23,
  # "Waluigi Stadium":23,
  # "waluigi stadium":23,
  # "Waluigi":23, 
  # "waluigi":23,
  # "DS Desert Hills":24,
  # "Desert Hills":24,
  # "desert hills":24,
  # "GBA Bowser's Castle 3":25,
  # "GBA 3":25,
  # "gba 3":25,
  # "BC3":25,
  # "bc3":25,
  # "N64 DK's Jungle Parkway":26,
  # "Parkway":26,
  # "parkway":26,
  # "GCN Mario Circuit":27,
  # "GCN mario circuit":27,
  # "GCN mario":27,
  # "gcn mario":27,
  # "gcn Mario":27,
  # "SNES Mario Circuit 3":28,
  # "SNES 3":28,
  # "snes 3":28,
  # "DS Peach Gardens":29,
  # "Peach Gardens":29,
  # "peach gardens":29,
  # "GCN DK Mountain":30,
  # "DK Mountain":30,
  # "dk mountain":30,
  # "mountain":30,
  # "Mountain":30,
  # "N64 Bowser's Castle":31,
  # "BC64":31,
  # "bc64":31,
  # "N64BC":31,
  # "n64bc":31,
  #  "Lava Lake":32}

  #Below is the logic for asking a user what they would like to do
  ########

  print('Welcome to Kartnite Stats v2.1')
  print('Below are the options, type the number of the option you would like')
  print('0. View Leaderboards \n1. Input a Race \n2. Edit A Singular Score \n3. View All Databases \n4. View Track Records \n5. View Player Stats \n6. Enter A Grand Prix Winner \n7. Enter Shock Dodges')

  selection = input('What Would You Like To Do: ')
  
  #selection logic
  
  #depending on the user selection it will ask for different inputs, and load the nessassary databases needed for that operation

  #input a race
  if selection == '1':
    print('Warining: track and racer names must be formatted correctally \nTrack names must match their full name \nRacer names must have their first letter capital')
    Track = input("Name of Track: ")
    Racers = input("Names of Racers: ")
    Scores = input("Scores:")
    #loading statment
    print("\nYour selection of", selection, "is loading...." )
    print('\n\n')
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    KartScore = sh.worksheet('Kart Score').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    inputRace(dfScores,dfRaceCount, dfKartScore, Track,Racers,Scores,TrackIndex)
 
 #edit a score
  elif selection == '2':
    print('Edit A Score is used to fix an error that may have occured while entering a race \nUse a positive number to increase a score, and a negative to decrease \nUse the net change you want to make as the value')
    Track = input("Name of Track: ")
    Racer = input("Names of Racer: ")
    Score = input("Score: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n\n')
    kartData = sh.worksheet('Total Scores').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    editAScore(dfScores, Track, Racer, Score,TrackIndex)
 
  #print all data
  elif selection == '3':
    print("\nYour selection of", selection, "is loading...." )
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    print('\nDatabase of total scores')
    print(dfScores)
    print('\nDatabase of Race Count')
    print(dfRaceCount)
    print('\nDatabase GP Wins')
    print(dfWins)
    print('\nDatabase of Shock Dodges')
    print(dfShock)
 
  #track data
  elif selection == '4':
    Track = input("Name of Track: ")
    print("\nYour selection of", selection, "is loading...." )
    print("\n\n")
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    GetTrackData(dfScores,dfRaceCount,Track,TrackIndex)
  
  #racer data
  elif selection == '5':
    Racer = input("Name of Racer: ")
    print("\nYour selection of", selection, "is loading...." )
    print("\n\n")
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    KartScore = sh.worksheet('Kart Score').get_all_values()
    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    getPlayerStats(dfScores, dfRaceCount, dfWins, dfShock, dfKartScore, Racer, TrackIndex)
  
  #enter a winner
  elif selection == '6':
    Player = input("Winning Player: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    Wins = sh.worksheet('GP Wins').get_all_values()
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    enterWinner(dfWins, Player)
  
  #enter a dodge
  elif selection == '7':
    Player = input("Player Name: ")
    Count = input("How Many Dodges: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    enterDodges(dfShock,Player,Count)
 
  #leaderbaords
  elif selection == '0':
    print("\nYour selection of", selection, "is loading...." )
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    KartScore = sh.worksheet('Kart Score').get_all_values()
    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    print('\n\n')
    print('Seeding Leaderboard')
    getSeedings(dfScores,dfRaceCount,dfWins,dfShock,dfKartScore,TrackIndex)
    print('\nTotal Races Played')
    getRaceCountLeaderbaords(dfRaceCount, TrackIndex)
    print('\nGP Wins')
    getGPWinsLeaderboard(dfWins)
    print('\nShock Dodges')
    getShockDodges(dfShock)

  else: 
    print('invalid selection restart the program')



  #####


  ##COMMENT THIS OUT WHEN TESTING

  #upadates the google sheet, a psyudo save of the data
  TotalScores = sh.worksheet('Total Scores')
  RaceCount = sh.worksheet('Race Count')
  WinCount = sh.worksheet('GP Wins')
  Shock = sh.worksheet('Shock Dodges')
  KartScore = sh.worksheet('Kart Score')
  
  #saves to the google sheet if nessassary, depending on the input of the user
  if selection =='1':
    set_with_dataframe(TotalScores, dfScores)
    set_with_dataframe(RaceCount,dfRaceCount)
    set_with_dataframe(KartScore,dfKartScore)
    print("Success - The Database Has Been Updated")
  elif selection =='2':
    set_with_dataframe(TotalScores, dfScores)
    print("Success - The Database Has Been Updated")
  elif selection =='6':
    set_with_dataframe(WinCount,dfWins)
    print("Success - The Database Has Been Updated")
  elif selection =='7':
    set_with_dataframe(Shock,dfShock)
    print("Success - The Database Has Been Updated")


In [112]:
#This is now the cell you need to run the program
#The old main cell was redefined from the main cell to the singular method below
#This is to reduce the amount of scrolling needed when inputing or viewing the data
######IMPORTS######
import pandas as pd
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
###################
#methodCall
RunKartniteStats()

Welcome to Kartnite Stats v2.1
Below are the options, type the number of the option you would like
0. View Leaderboards 
1. Input a Race 
2. Edit A Singular Score 
3. View All Databases 
4. View Track Records 
5. View Player Stats 
6. Enter A Grand Prix Winner 
7. Enter Shock Dodges
What Would You Like To Do: 4
Name of Track: Toad's Factory

Your selection of 4 is loading....



Stats For:  Toad's Factory

 Total Scores
  Tracks x Players Demitri Pat Chris Joe Shane Mike Kevin Matt
3   Toad's Factory      54  54    36  22     0   10    25    0

Times Played: 18

Player Averages:
Kevin :  12.5
Demitri :  13.5
Mike :  10.0
Chris :  9.0
Matt :  0
Pat :  13.5
Shane :  0
Joe :  7.333333333333333


Demitri, Pat is the MVP for this track

MVP Leaderboard
    Player  MVP Points
1  Demitri   40.365672
5      Pat   40.365672
3    Chris   26.910448
0    Kevin   24.937811
7      Joe   18.278607
2     Mike   14.975124
4     Matt    0.000000
6    Shane    0.000000
